# PLC 원본 데이터 정규화
PLC 원본 데이터는 raw데이터이기 때문에 클린작업이 필요.

** PLC 원본 데이터 컬럼 **
- _time, _raw
위 두 컬럼 중 _time은 수집 시간, _raw는 해당 PLC의 데이터 의미.

** '_raw' 컬럼의 데이터를 분석, 분류하여 분석을 위한 데이터로 변환 **

* '_raw' 컬럼의 데이터 분류
    * Time, Tag, Value, Quality 로 이루어짐.
    * 그 중에서 Tag의 내용은 4개의 값이 합쳐서 이루어짐.('.'으로 구분되어 있음)
    * Tag의 값을 OPCSever, LSE, Address, Equipment_id로 구분하여 분리.
    * '_raw' 컬럼은 총 7가지 컬럼으로 분류(분리). 
    * time 컬럼 부분은 포멧이 일치히지 않아 시간 포멧형태로 변경함.('+0000' 부분을 시간뒤에 붙이도록 함.)
* '_raw' 컬럼 내용중 value와 tag 값이 비정상적인 값이 존재
    * 해당 값은 임의 값으로 처리.
* 데이터 중 값이 일부만 존재하는 행이 존재.
    * 해당 행은 제외하여 처리

In [19]:
import csv
import re
from datetime import datetime
import dateutil.parser
import codecs
import glob

# 원본 데이터 경로
working_dir = "./"

# 변경한 파일 경로
reg_working_dir="./reg_files/"

# 원본 데이터 파일 리스트 불러오기
plc_data_list = glob.glob(working_dir+"PLC_DATA*")

In [15]:
# 파일에서 데이터 읽어서 파싱하는 함수
def read_files(file_name):
    data=[]

    with codecs.open(file_name, 'r', encoding="utf-8-sig") as csvfile:#파일 오픈시, 처음 값이 이상한 값이 나오기 때문에 인코딩을 지정함.
        spamreader = csv.DictReader(csvfile)
        count = 0

        for row in spamreader:
            count+=1
            raw = row["_raw"]
            raw_split = raw.split(' ')
            
            if len(raw_split) > 5:# 데이터 중에서 일부 데이터가 없거나 비정상적인 값 때문에 
                                  # 분리한 _raw 값이 5가지 이상이 될 경우 수행
                # 데이터 수집 시간
                _time = row["_time"]
                
                # 데이터 생성 시간
                time = ' '.join(raw_split[0:2])
                time = ''.join([time,raw_split[2]])
                
                # 시간 데이터 시간 포멧으로 파싱
                # 시간 포멧으로 변경하면 시간이 오래걸림.
                # 파일 수정작업에서는 일반적으로 문자열로 처리하여 작업 진행함.
                #_time = dateutil.parser.parse(row["_time"])
                #time = dateutil.parser.parse(time)
                
                # Parse Value(value 파싱)
                try:
                    m = re.search('(?<=Value=)\"\w+\"', raw_split[4])
                    value = m.group(0).split("\"")[1]
                except:
                    #print("ERROR_VALUE")
                    value = '0'#일부 데이터에서 값이 존재하지 않아 임의로 0을 할당.
                    
                # Parse Quality
                try:
                    m = re.search('(?<=Quality=)\"\w+\"', raw_split[5])
                    quality = m.group(0).replace("\"","")
                except:
                    #print("ERROR_QUALITY")
                    quality = "" # 값이 없으면 빈칸 처리

                # Parse Tag 
                try:
                    # 테그 값 분리('.' 기준으로 분리)
                    tag = raw_split[3].split("\"")[1]
                    tag_split = tag.split('.')
                    # 분리된 값, 각 변수에 배정.
                    OPCServer = tag_split[0]
                    lse = tag_split[1]
                    address = tag_split[3].replace("%","")
                    equipment_id = tag_split[2]
                except:
                    print(row)
                
                data_format={"_time":_time, 
                             "time":time, 
                             "opcserver":OPCServer, 
                             "lse":lse, 
                             "address":address, 
                             "equipment_id":equipment_id, 
                             "value":value, 
                             "quality":quality}
                data.append(data_format)
        
        print("Total Rows : ",count)# 작업한 행의 수
    return data

In [16]:
# 정규화된 내용을 csv파일로 저장하는 함수.
def writer_reg(file_name, data):
    with open(file_name,'w',newline='') as wcsv:
        fieldnames = ["_time", "time", "opcserver", "lse", "address", "equipment_id", "value", "quality"]
        writer = csv.DictWriter(wcsv, delimiter=',',fieldnames=fieldnames)

        writer.writeheader()
        # 데이터 쓰기 작업, wrier
        writer.writerows(data)# 여러줄을 한번에 쓰는 함수

        #for each in data:
        #writer.writerow(each)# 한 줄씩 쓰는 함수

In [20]:
#원본 파일을 불러와 분석하여 정규화(클린?)작업하여 새로운 파일로 저장.
start_total = datetime.now()
for each_file in plc_data_list:
    start = datetime.now()
    print(each_file)# show each file name 
    
    _data = read_files(each_file)
    
    w_file_name = each_file[2:-4]+"_reg.csv" # 저장 파일 이름 변경.
    writer_reg(reg_working_dir+w_file_name, _data)
    
    finish = datetime.now()
    print("Finish Time : ", (finish-start), "sec") #파일 하나의 변환 경과 시간

finish_total = datetime.now()
print("Finished\n, Total time : ", finish_total- start_total, "sec")

.\PLC_DATA_20160106-20160108.csv
Total Rows :  233039
Finish Time :  0:00:10.403479 sec
.\PLC_DATA_20160109-20160114.csv
Total Rows :  17813
Finish Time :  0:00:00.920713 sec
.\PLC_DATA_20160115-20160118.csv
Total Rows :  128122
Finish Time :  0:00:05.704708 sec
.\PLC_DATA_20160119-20160121.csv
Total Rows :  287102
Finish Time :  0:00:14.252156 sec
.\PLC_DATA_20160122-20160126.csv
Total Rows :  255144
Finish Time :  0:00:11.999303 sec
.\PLC_DATA_20160127-20160131.csv
Total Rows :  234319
Finish Time :  0:00:10.432713 sec
.\PLC_DATA_20160201-20160207.csv
Total Rows :  81833
Finish Time :  0:00:03.556802 sec
.\PLC_DATA_20160208-20160216.csv
Total Rows :  290355
Finish Time :  0:00:12.514619 sec
.\PLC_DATA_20160217-20160222.csv
Total Rows :  283698
Finish Time :  0:00:12.191571 sec
.\PLC_DATA_20160223-20160225.csv
Total Rows :  265305
Finish Time :  0:00:11.621538 sec
.\PLC_DATA_20160226-20160229.csv
Total Rows :  53992
Finish Time :  0:00:02.291660 sec
.\PLC_DATA_20160301-20160303.csv
To